# Obtaining Wildfire Data

This notebook is an adaptation of a notebook written by Dr. David W. McDonald (referenced in cell below) for the DATA 512 cource part of the UW MS Data Science program. Except for the final few cells and a few lines, this notebook is almost a direct copy from Professor McDonald's notebook, largely due to the fact it was leveraged with reference code to acquire wildfire data and read the data in an efficient manner.

## Assumptions
1. The `GeoJSON Files.zip` file under `Attached Files` from the [following link](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) is downloaded. Unzipping that file will reveal another file `USGS_Wildland_Fire_Combined_Dataset.json` which is used in this notebook. It is assumed the location of that final `.json` file is at the same level as this notebook.
2. The `wildfire` directory with `__init__.py` and `Reader.py` exists and is at the same level as this notebook. The code from `Reader.py` will be used in this notebook and imported.
3. The following cells have been read and understood as it relates to the dataset being downloaded, License, etc.

## Goals
This notebook is focused on obtaining specific features of each fire from the wildfire dataset `USGS_Wildland_Fire_Combined_Dataset.json`. Each of the desired features are saved alongside a distance calculation of each fire from the specific city of Gainesville, Florida. The specifics of the distance computed is detailed further down in the notebook.

## Final Outputs
A `.pkl` object by the name of `wildfire.pkl` is saved as a data product from this notebook. The produced data product is used by the `smoke_metric.ipynb` notebook. Run this notebook first, followed by `aqi.ipynb`, then finally `smoke_metric.ipynb`.

# Wildfire Proximity Computation Example
This Jupyter notebook contains example code that illustrate how to perform some basic geodetic computations related to the Wildfire course project. The notebook is structured as a set of examples that illustrate something about the the data schema or illustrate a way to compute specific values. This notebook is not a tutorial on performing geodetic computations, but illustrates a number of key concepts. This notebook should provide enough information to complete the Wildfire assignment.

The complete [Wildfire dataset](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) can be retrieved from a US government repository. I have noticed that the repository is sometimes "down" and does not respond. It probably makes sense to get the dataset as soon as possible.

This notebook has dependencies on [Pyproj](https://pyproj4.github.io/pyproj/stable/index.html), the [geojson](https://pypi.org/project/geojson/) module and on the wildfire user module. Pyproj and geojson can be installed via pip. The wildfire user module should be downloaded from the course website, unzipped, and moved into the folder pointed to by your PYTHONPATH system variable.

### License
This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.1 - August 16, 2024

### Preliminaries
First we start with some imports and some constant definitions.

In [1]:
#
#    IMPORTS
# 

#    Import some standard python modules
import os, json, time
#
#    The module pyproj is a standard module that can be installed using pip or your other favorite
#    installation tool. This module provides tools to convert between different geodesic coordinate systems
#    and for calculating distances between points (coordinates) in a specific geodesic system.
#
from pyproj import Transformer, Geod
#
#    The 'wildfire' module is a user module. This module is available from the course website. The module
#    includes one object, a Reader, that can be used to read the GeoJSON files associated with the
#    wildefire dataset. The module also contains a sample datafile that is GeoJSON compliant and that
#    contains a small number of wildfires extracted from the main wildfire dataset.
#    
from wildfire.Reader import Reader as WFReader
#

#    
import geojson
#

In [2]:
# This file must be obtained from the USGS source mentioned above
EXTRACT_FILENAME = "./USGS_Wildland_Fire_Combined_Dataset.json"

#
#    The user module 'wildfire' contains a Reader object and the sample data. This bit of code finds where that is
#    located on your machine and constructs a path so that the sample data can be loaded. This assumes you have set 
#    a PYTHONPATH environment variable to point to the location on your machine where you store python user modules.
#
#    NOTE: if you use Anaconda for virtual python environments, Anaconda will adhere to the PYTHONPATH conventions
#    for user modules.
#
MODULENAME = "wildfire"
MODULEPATH = ""
try:
    ppath = os.environ.get('PYTHONPATH')
    if not ppath: raise
    MODULEPATH = os.path.join(ppath,MODULENAME)
except:
    # Likely here because a PYTHONPATH was not set, show a warning message
    print("Looks like you're not using a 'PYTHONPATH' to specify the location of your python user modules.")
    print("You may have to modify the sample code in this notebook to get the documented behaviors.")
    MODULEPATH = ""

if MODULEPATH:
    SAMPLE_DATA_FILENAME = os.path.join(MODULEPATH,EXTRACT_FILENAME)
else:
    SAMPLE_DATA_FILENAME = EXTRACT_FILENAME
#
# print out where we think we're going to find the sample data
print(f"{SAMPLE_DATA_FILENAME=}")

#
#    A dictionary of some city locations from the US west coast states.
#
CITY_LOCATIONS = {
    'anchorage' :     {'city'   : 'Anchorage',
                       'latlon' : [61.2176, -149.8997] },
    'ocean_shores' :  {'city'   : 'Ocean Shores',    
                       'latlon' : [47.0074, -124.1614] },
    'seaside' :       {'city'   : 'Seaside',
                       'latlon' : [45.9932, -123.9226] }, 
    'bend' :          {'city'   : 'Bend',
                       'latlon' : [44.0582, -121.3153] }, 
    'medford' :       {'city'   : 'Medford',
                       'latlon' : [42.3265, -122.8756] }, 
    'crescent_city' : {'city'   : 'Crescent City',
                       'latlon' : [41.7558, -124.2026] }, 
    'tomales' :       {'city'   : 'Tomales',
                       'latlon' : [38.2411, -122.9033] }, 
    'barstow' :       {'city'   : 'Barstow',
                       'latlon' : [34.8958, -117.0173] }, 
    'redding' :       {'city'   : 'Redding',
                       'latlon' : [40.5865, -122.3916] }, 
    'encinitas' :     {'city'   : 'Encinitas',
                       'latlon' : [33.0370, -117.2920] },
    'loveland' :      {'city'   : 'Loveland',
                       'latlon' : [40.398857, -105.052643] },
    'gainesville':    {'city': 'Gainesville',
                       'latlon': [29.68, -82.35] }
}


Looks like you're not using a 'PYTHONPATH' to specify the location of your python user modules.
You may have to modify the sample code in this notebook to get the documented behaviors.
SAMPLE_DATA_FILENAME='./USGS_Wildland_Fire_Combined_Dataset.json'


## Example 1. Load the wildfire data using the GeoJSON module

In this example we use the GeoJSON module ([documentation](https://pypi.org/project/geojson/), [GitHub repo](https://github.com/jazzband/geojson)) to load the sample file. This module works mostly the way you would expect. GeoJSON is mostly just JSON, so actually, you don't even really need to use the GeoJSON module. However, that module will do some conversion of Geo type things to something useful. However, this example, and the examples that follow, do not rely on specific Geo features from geojson.


In [3]:
#
#    Open a file, load it with the geojson loader
#
print(f"Attempting to open '{SAMPLE_DATA_FILENAME}'")
geojson_file = open(SAMPLE_DATA_FILENAME,"r")
print(f"Using GeoJSON module to load sample file '{SAMPLE_DATA_FILENAME}'")
gj_data = geojson.load(geojson_file)
geojson_file.close()
#
#    Print the keys from the object
#
gj_keys = list(gj_data.keys())
print("The loaded JSON dictionary has the following keys:")
print(gj_keys)
print()
#
#    For all GeoJSON type things, the most important part of the file are the 'features'. 
#    In the case of the wildfire dataset, each feature is a polygon (ring) of points that define the bounary of a fire
#
count = 0
for feature in gj_data['features']:
    count += 1
    #print(json.dumps(feature,indent=4))
    #time.sleep(0.5)   # this slows the output to fix output rate limits for Jupyter

print(f"Found {count} features in the variable 'gj_data' ")
#

Attempting to open './USGS_Wildland_Fire_Combined_Dataset.json'
Using GeoJSON module to load sample file './USGS_Wildland_Fire_Combined_Dataset.json'
The loaded JSON dictionary has the following keys:
['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields', 'features']

Found 135061 features in the variable 'gj_data' 


In [4]:
#
#    Get the first item in the list of features
#
SLOT = 6
gj_feature = gj_data['features'][SLOT]
#
#    Print everyting in this dictionary (i.e., gj_feature) - it's long
#
print(f"The wildfire feature from slot '{SLOT}' of the loaded gj_data['features']")
print(json.dumps(gj_feature, indent=4))


The wildfire feature from slot '6' of the loaded gj_data['features']
{
    "attributes": {
        "OBJECTID": 7,
        "USGS_Assigned_ID": 7,
        "Assigned_Fire_Type": "Wildfire",
        "Fire_Year": 1878,
        "Fire_Polygon_Tier": 1,
        "Fire_Attribute_Tiers": "1 (2), 3 (4)",
        "GIS_Acres": 39187.07687859968,
        "GIS_Hectares": 15858.447374124367,
        "Source_Datasets": "Comb_National_NIFC_Interagency_Fire_Perimeter_History (2), Comb_National_WFDSS_Interagency_Fire_Perimeter_History (1), Comb_National_BLM_Fire_Perimeters_LADP (1), Comb_SubState_BLM_Idaho_NOC_FPER_Historica_Fire_Polygons (1), Comb_National_USFS_Final_Fire_Perimeter (1)",
        "Listed_Fire_Types": "Wildfire (3), Likely Wildfire (3)",
        "Listed_Fire_Names": "1878 (6)",
        "Listed_Fire_Codes": "No code provided (6)",
        "Listed_Fire_IDs": "1878-NA-000000 (2), 1878-IDNCF- (2)",
        "Listed_Fire_IRWIN_IDs": "",
        "Listed_Fire_Dates": "Listed Other Fire Date(s): 201

In [15]:
#
#    Every feature has a 'geometry' which specifies geo coordinates that make up each geographic thing
#    In the case of the wildfire data, most wildfires are bounded shapes, circles, squares, etc. This is
#    represented by shapes called 'rings' in GeoJSON.
# 
# Get the geometry for the feature we pulled from the feature_list
gj_geometry = gj_feature['geometry']
# The largest shape (ring) is supposed to be item zero in the list of 'rings'
gj_bigest_ring = gj_geometry['rings'][0]

print(f"The largest ring of gj_feature['features'][{SLOT}]['rings'] consists of {len(gj_bigest_ring)} points.")

The largest ring of gj_feature['features'][6]['rings'] consists of 879 points.


## Example 2. Load the wildfire data using the wildfire Reader object

In the following cells we provide small code snippets that do the following:
- Create a wildfire Reader() object and use it to open the sample data file. Once, opened, we have access to the header information so we print that to show the structure of that data.
- Use the Reader() object and the next() method to read a set of wildfire features. The small sample file should have 15 of them. 
- Print one example feature showing the dictionary data structure of a feature.
- Access the geometry of one specific feature to get the 'ring' boundary of that specific fire - which is a list of geodetic coordinates.

Note that some of the output cells are quite long. Once you understand what they are illustrating you might want to collapse them or comment out the print statements that generate the output.

Another note regarding terminology. In the GeoJSON standard, something that is to be represented geographically is generically called a 'feature'. In the case of the wildfire dataset every 'feature' is a wildfire. These terms are used somewhat interchangably below.


In [7]:
#
#    This bit of code opens a new wildfire reader, gets the header information and prints it to the screen
#
print(f"Attempting to open '{SAMPLE_DATA_FILENAME}' with wildfire.Reader() object")
wfreader = WFReader(SAMPLE_DATA_FILENAME)
print()
#
#    Now print the header - it contains some useful information
#
header_dict = wfreader.header()
header_keys = list(header_dict.keys())
print("The header has the following keys:")
print(gj_keys)
print()
print("Header Dictionary")
print(json.dumps(header_dict,indent=4))


Attempting to open './USGS_Wildland_Fire_Combined_Dataset.json' with wildfire.Reader() object

The header has the following keys:
['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields', 'features']

Header Dictionary
{
    "displayFieldName": "",
    "fieldAliases": {
        "OBJECTID": "OBJECTID",
        "USGS_Assigned_ID": "USGS Assigned ID",
        "Assigned_Fire_Type": "Assigned Fire Type",
        "Fire_Year": "Fire Year",
        "Fire_Polygon_Tier": "Fire Polygon Tier",
        "Fire_Attribute_Tiers": "Fire Attribute Tiers",
        "GIS_Acres": "GIS_Acres",
        "GIS_Hectares": "GIS_Hectares",
        "Source_Datasets": "Source Datasets",
        "Listed_Fire_Types": "Listed Fire Types",
        "Listed_Fire_Names": "Listed Fire Names",
        "Listed_Fire_Codes": "Listed Fire Codes",
        "Listed_Fire_IDs": "Listed Fire IDs",
        "Listed_Fire_IRWIN_IDs": "Listed Fire IRWIN IDs",
        "Listed_Fire_Dates": "Listed Fire Dates",
        

In [8]:
#
#    This sample code will load the whole sample (extracted data) file, or a small amount of the complete dataset.
#
MAX_FEATURE_LOAD = 200000
feature_list = list()
feature_count = 0
# A rewind() on the reader object makes sure we're at the start of the feature list
# This way, we can execute this cell multiple times and get the same result 
wfreader.rewind()
# Now, read through each of the features, saving them as dictionaries into a list
feature = wfreader.next()
while feature:
    feature_list.append(feature)
    feature_count += 1
    # if we're loading a lot of features, print progress
    if (feature_count % 100) == 0:
        print(f"Loaded {feature_count} features")
    # loaded the max we're allowed then break
    if feature_count >= MAX_FEATURE_LOAD:
        break
    feature = wfreader.next()
#
#    Print the number of items (features) we think we loaded
print(f"Loaded a total of {feature_count} features")
#
#    Just a validation check - did all the items we loaded get into the list?
print(f"Variable 'feature_list' contains {len(feature_list)} features")




Loaded 100 features
Loaded 200 features
Loaded 300 features
Loaded 400 features
Loaded 500 features
Loaded 600 features
Loaded 700 features
Loaded 800 features
Loaded 900 features
Loaded 1000 features
Loaded 1100 features
Loaded 1200 features
Loaded 1300 features
Loaded 1400 features
Loaded 1500 features
Loaded 1600 features
Loaded 1700 features
Loaded 1800 features
Loaded 1900 features
Loaded 2000 features
Loaded 2100 features
Loaded 2200 features
Loaded 2300 features
Loaded 2400 features
Loaded 2500 features
Loaded 2600 features
Loaded 2700 features
Loaded 2800 features
Loaded 2900 features
Loaded 3000 features
Loaded 3100 features
Loaded 3200 features
Loaded 3300 features
Loaded 3400 features
Loaded 3500 features
Loaded 3600 features
Loaded 3700 features
Loaded 3800 features
Loaded 3900 features
Loaded 4000 features
Loaded 4100 features
Loaded 4200 features
Loaded 4300 features
Loaded 4400 features
Loaded 4500 features
Loaded 4600 features
Loaded 4700 features
Loaded 4800 features
L

In [9]:
#
#    The 'feature_list' variable was created when we read the sample file in a code cell above
#    Now, we're just going to look at one single feature - see what is in there
#

SLOT = 0
wf_feature = feature_list[SLOT]

# Print everyting in this dictionary (i.e., wf_feature) - it's long
print(f"The wildfire feature from slot '{SLOT}' of the loaded 'feature_list'")
print(json.dumps(wf_feature, indent=4))


The wildfire feature from slot '0' of the loaded 'feature_list'
{
    "attributes": {
        "OBJECTID": 1,
        "USGS_Assigned_ID": 1,
        "Assigned_Fire_Type": "Wildfire",
        "Fire_Year": 1860,
        "Fire_Polygon_Tier": 1,
        "Fire_Attribute_Tiers": "1 (1)",
        "GIS_Acres": 3940.20708940724,
        "GIS_Hectares": 1594.5452365353703,
        "Source_Datasets": "Comb_National_NIFC_Interagency_Fire_Perimeter_History (1)",
        "Listed_Fire_Types": "Wildfire (1)",
        "Listed_Fire_Names": "Big Quilcene River (1)",
        "Listed_Fire_Codes": "No code provided (1)",
        "Listed_Fire_IDs": "",
        "Listed_Fire_IRWIN_IDs": "",
        "Listed_Fire_Dates": "Listed Other Fire Date(s): 2006-11-02 - NIFC DATE_CUR field (1)",
        "Listed_Fire_Causes": "",
        "Listed_Fire_Cause_Class": "Undetermined (1)",
        "Listed_Rx_Reported_Acres": null,
        "Listed_Map_Digitize_Methods": "Other (1)",
        "Listed_Notes": "",
        "Processing

In [10]:
#
#    Every feature has a 'geometry' which specifies geo coordinates that make up each geographic thing
#    In the case of the wildfire data, most wildfires are bounded shapes, circles, squares, etc. This is
#    represented by shapes called 'rings' in GeoJSON.
# 
# Get the geometry for the feature we pulled from the feature_list
wf_geometry = wf_feature['geometry']
# The largest shape (ring) is supposed to be item zero in the list of 'rings'
wf_bigest_ring = wf_geometry['rings'][0]

print(f"The largest ring of wf_feature['features'][{SLOT}]['rings'] consists of {len(wf_bigest_ring)} points.")

The largest ring of wf_feature['features'][0]['rings'] consists of 768 points.


## Example 3. Distance computations with Pyproj

One issue in performing geodetic computation is that any (all) geographic coordinate systems are eventually translated to the surface of the earth - which is not flat. That means every computation of distance between two points is some kind of arc (not actually a straight line). Further the earth is not a true sphere, its a type of ellipsoid. That means the amount of curvature varies depending upon where you are on the surface and the direction - which changes the distance.

Lucky for us there are geographers who like to write code and have built systems to simplify the computation of distances over the earth's surface. One of those systems is called [Pyproj](https://pyproj4.github.io/pyproj/stable/index.html). It has functions that will convert coordinate points between (almost) any two of the many different geographic coordinate systems. As well, Pyproj provides ways to compute distances between two points (mostly assuming the points are already in the same coordinate system).

This example uses the Geod() object to calculate the distance between a slected starting city and all of the cities defined in our CITY_LOCATIONS dictionary (see CONSTANTS above).

The example calls the distances computed 'straight line' distances - because that is what you would have to use to find the distance between two cities using Google. If you do not use that terminology as part of your search, Google will map roads to get you between a source and destination; that will never match our calculation.


In [11]:
#
#    First create a geodesic model that will be used for the calculations. There are a number of
#    different models of the earth. The WSG84 is one that is commonly used and relatively up-to-date
#
#geodcalc = Geod(ellps='clrk66')       # Use Clarke 1866 ellipsoid representation of the earth
geodcalc = Geod(ellps='WGS84')         # Use WGS84 ellipsoid representation of the earth

#    Two constants for accessing the 'latlon' array in our CITY_LOCATIONS constant dict
LAT = 0
LON = 1
#    Get a city from our CITY_LOCATIONS constant as our starting position
start_at = CITY_LOCATIONS["medford"]
start_at = CITY_LOCATIONS["redding"]
start_at = CITY_LOCATIONS["loveland"]

#    Loop through all of the cities to calculate the distance from the starting position
for city_key in CITY_LOCATIONS.keys():
    #    City destination
    destination = CITY_LOCATIONS[city_key]
    #    Note that the 'inv()' function wants coordinates in Longitude,Latitude order by default
    #    inv() also allows lat and lon parameters to be vectors/arrays - in which case the results would be vector/arrarys
    distance = geodcalc.inv(start_at['latlon'][LON],start_at['latlon'][LAT],destination['latlon'][LON],destination['latlon'][LAT])
    #    The 'distance' result variable is a tuple/list with the first two items reflecting forward/backward azimuths
    #    and the third item representing the distance in meters. 
    d_meters = distance[2]
    d_miles = d_meters * 0.00062137 # constant to convert meters to miles
    #    BTW, this isn't actually a 'straight' line because the whole reason for using pyproj is to calculate
    #    these distance measures over the surface of a sphere/ellipsoid. We set up which ellipsoid to use when we
    #    defined the 'geodcalc' object near the top of this cell
    print(f"Straight line distance from {start_at['city']} to {destination['city']} is {d_meters} meters or {d_miles:5.2f} miles")



Straight line distance from Loveland to Anchorage is 3800330.933788066 meters or 2361.41 miles
Straight line distance from Loveland to Ocean Shores is 1700334.6351250918 meters or 1056.54 miles
Straight line distance from Loveland to Seaside is 1649822.3613664224 meters or 1025.15 miles
Straight line distance from Loveland to Bend is 1400008.5319638762 meters or 869.92 miles
Straight line distance from Loveland to Medford is 1503732.3001080346 meters or 934.37 miles
Straight line distance from Loveland to Crescent City is 1612979.0529373728 meters or 1002.26 miles
Straight line distance from Loveland to Tomales is 1555123.1658472163 meters or 966.31 miles
Straight line distance from Loveland to Barstow is 1218224.6806120276 meters or 756.97 miles
Straight line distance from Loveland to Redding is 1467713.6260297254 meters or 911.99 miles
Straight line distance from Loveland to Encinitas is 1362736.6411465146 meters or 846.76 miles
Straight line distance from Loveland to Loveland is 0.0

## Example 4. Convert points between geodetic coordinate systems

One of the constraints in doing geodetic computations is that most of the time we need to have our points (the coordinates for places) in the same geographic coordinate system. There are tons and tons of coordinate systems. You can find descriptions of many of them at [EPSG.io](https://epsg.io).

Looking at the wildfire header information, you can find this in the output of Example 1 (above), we can see fields named "geometryType" and "spatialReference". This looks like:

        "geometryType": "esriGeometryPolygon",
        "spatialReference": {
            "wkid": 102008,
            "latestWkid": 102008
        },

This indicates that the geometry of our wildfire data are generic polygons and that they are expressed in a coordinate system with the well-known ID (WKID) 102008. This coordinate system is also known as [ESRI:102008](https://epsg.io/102008)

If you look back at Example 2 (above), you might have wondered about the line of code that says:

    geocalc = Geod(ellps='WGS84')         # Use WGS84 ellipsoid representation of the earth

That string, 'WGS84', is a representation of the earth, that also relies on a well known coordinate system that is sometimes called 'decimal degrees' (DD). That decimal degrees system has an official name (or WKID) of [EPSG:4326](https://epsg.io/4326).

For the example below, what we're going to do is take the geometry of a fire feature, extract the largest ring (i.e., the largest boundary of the fire) and convert all of the points in that ring from the ESRI:102008 coordinate system to EPSG:4326 coordinates.

Why? You might ask. Well, it turns out that the easy way to get locations for a city is to make a Google query for the location. That will respond with a decimal degress version of latitued and longitude. So, this allows us to move everything into decimal degress before we perform a distance calcluation.


In [12]:
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

In [13]:
#
#   Convert one ring from the default to EPSG
#
#   There are two options here - depending upon whether you loaded data useing GeoJSON or the wildfire.Reader
#
#ring_in_epsg4326 = convert_ring_to_epsg4326(gj_bigest_ring)
#
ring_in_epsg4326 = convert_ring_to_epsg4326(wf_bigest_ring)
#
print(f"Ring consists of {len(ring_in_epsg4326)} points.")
#
#    If you want to print them out you can see what they look like converted.
#print(ring_in_epsg4326)
#for point in ring_in_epsg4326:
#    print(f"{point[0]},{point[1]}")


Ring consists of 768 points.


## Example 5. Compute distance between a place and a wildfire

The basic problem is knowing how far away a fire is from some location (like a city). One issue is that fires are irregularly shaped so the actual answer to that is a bit dependent upon the exact shape and how you want to think about the notion of 'distance'. For example, should we just find the closest point on the perimiter of a fire and call that the distance? Maybe we should find the centroid of the region, identify that as a geolocation (coordinate) and then calculate the distance to that? We can come up with numerous other ways.

The first bit of code finds the point on the perimiter with the shortest distance to the city (place) and returns the distance as well as the lat,lon of the perimeter point.

The second bit of code calculates the average distance of all perimeter points to the city (place) and returns that average as the distance. This is not quite what the centroid would be, but it is probably fairly close.

These are two reasonable ways to think about possible distance to a fire. But both require computing distance to a whole set of points on the perimeter of a fire.


In [14]:
#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point



#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns the average miles from boundary to the place
#
def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])
    #print("Got the following list:",distances_in_meters)
    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    return average



## Data Product Creation

Below is the filtering of the full Wildfire Dataset to create a secondary data product that will be further transformed in the `smoke_metric.ipynb` notebook for final analysis.

The following attributes are being extracted for every fire:
1. Wildfire name
2. Wildfire year (what year the fire happened)
3. Wildfire size (in acres)
4. Wildfire type
5. Wildfire Ring (Polygon object describing perimeter of fire)

A distance calculation is made from the fire to Gainesville, FL. The chosen way to compute distance is by computing an "average" distance from every perimeter point described by the `Shape` object of the Wildfire Ring. So, the latitude and longitude of Gainesville, FL is used against the Latitude and Longitude of a Polygon point to compute a distance. This type of distance is computed for every point and then average. This "distance" is saved alongside all of the above extracted features.

Finally, all wildfires before 1961 are discarded due to the advent of satillite imagery.

In [ ]:
final_data_list = []
count = 0
bad_features = []
place = CITY_LOCATIONS["gainesville"]

for i, wf_feature in enumerate(feature_list):
    data_dict = {}
    #print(f"{place['city']}")
    wf_year = wf_feature['attributes']['Fire_Year']
    if wf_year < 1961:
        continue
    wf_name = wf_feature['attributes']['Listed_Fire_Names'].split(',')[0]
    wf_size = wf_feature['attributes']['GIS_Acres']
    wf_type = wf_feature['attributes']['Assigned_Fire_Type']
    if count >= 93097:
        pass
    elif final_data_list[count]['name'] == wf_name:
        print(count)
        count += 1
        continue
    if 'rings' in wf_feature['geometry']:
        ring_data = wf_feature['geometry']['rings'][0]
    elif 'curveRings' in wf_feature['geometry']:
        ring_data = wf_feature['geometry']['curveRings'][0]
    else:
        raise Exception("HEY! No compatible geometry in this fire data!!!")
    data_dict['year'] = wf_year
    data_dict['name'] = wf_name
    data_dict['size'] = wf_size
    data_dict['type'] = wf_type
    data_dict['ring'] = ring_data
    # Use try except block in case of unexpected data that may cause problems
    try:
        # Compute using the average distance to all points on the perimeter
        distance = average_distance_from_place_to_fire_perimeter(place['latlon'],ring_data)
    except Exception as e:
        bad_features.append((i, wf_feature))
        continue
    data_dict['distance'] = distance
    final_data_list.append(data_dict)

In [26]:
print(i)
print(count)
feature_list[i]

109604
93097


{'attributes': {'OBJECTID': 109605,
  'USGS_Assigned_ID': 109605,
  'Assigned_Fire_Type': 'Prescribed Fire',
  'Fire_Year': 2007,
  'Fire_Polygon_Tier': 7,
  'Fire_Attribute_Tiers': '7 (2)',
  'GIS_Acres': 1216.3215961699473,
  'GIS_Hectares': 492.2278863164171,
  'Source_Datasets': 'Comb_National_USFS_Haz_Fuels_Prescribed_Fire (1), Comb_State_Rx_Only_Cal_Fire_Rx_Prescribed_Fire (1)',
  'Listed_Fire_Types': 'Prescribed Fire (2)',
  'Listed_Fire_Names': 'BARNES SOUTH UNDERBURN - BARNES SOUTH  UNDERBURN (1), BARNES SOUTH UNDERBURN (1)',
  'Listed_Fire_Codes': 'No code provided (2)',
  'Listed_Fire_IDs': '0515524052001000000 (2)',
  'Listed_Fire_IRWIN_IDs': None,
  'Listed_Fire_Dates': 'Listed Prescribed Fire Start Date(s): 2006-12-01 (1) | Listed Prescribed Fire End Date(s): 2007-02-14 (2) | Listed Upload Date(s): 2007-06-29 (1)',
  'Listed_Fire_Causes': 'Human (2)',
  'Listed_Fire_Cause_Class': 'Human (2)',
  'Listed_Rx_Reported_Acres': '1185.0 (1), 1203.34238154283 (1)',
  'Listed_Map_

In [34]:
with open('wildfire.pkl', 'wb') as f:
    pickle.dump(final_list, f)